<a href="https://colab.research.google.com/github/MasoudShaker/Aligorithm-Design-Assignments/blob/main/Copy_of_speech_command_classification_with_torchaudio_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline


# Speech Command Classification with torchaudio

This tutorial will show you how to correctly format an audio dataset and
then train/test an audio classifier network on the dataset.

Colab has GPU option available. In the menu tabs, select “Runtime” then
“Change runtime type”. In the pop-up that follows, you can choose GPU.
After the change, your runtime should automatically restart (which means
information from executed cells disappear).

First, let’s import the common torch packages such as
[torchaudio](https://github.com/pytorch/audio)_ that can be installed
by following the instructions on the website.


In [3]:
!pip install pydub

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm
import wave
import os
from pydub import AudioSegment

Let’s check if a CUDA GPU is available and select our device. Running
the network on a GPU will greatly decrease the training/testing runtime.




In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Connect to Google Drive

In [6]:
######### Masoud cell #########
from google.colab import drive
drive.mount('drive')

Mounted at drive


## Importing the Dataset


In [9]:
######### Masoud cell #########
labels = []
Bale_path = 'drive/MyDrive/Bale'
folder_names = os.listdir(Bale_path)
train_set_list = []
test_set_list = []
split_percentage = 0.9

for idx, folder_name in enumerate(folder_names):

  sample_list = []
  label = folder_name[2:]
  labels.append(label)

  folder_path = os.path.join(Bale_path, folder_name)
  file_names = os.listdir(folder_path)
  # print(file_names)
  num_files = len(file_names)

  for i, file_name in enumerate(file_names):
    file_path = os.path.join(folder_path, file_name)

    stereo_audio = AudioSegment.from_file(
    file_path,
    format="wav")
    mono_audios = stereo_audio.split_to_mono()

    file_idx = idx*10 + i

    mono_left_path = f"drive/MyDrive/mono_lefts/mono_left{file_idx}.wav"
    mono_right_path = f"drive/MyDrive/mono_rights/mono_right{file_idx}.wav"

    mono_left = mono_audios[0].export(mono_left_path, format="wav")

    mono_right = mono_audios[1].export(mono_right_path, format="wav")

    # metadata_left = torchaudio.info(file_path_left)
    waveform_left, sample_rate = torchaudio.load(mono_left_path)
    # print(f'mono left \n waveform: {waveform_left} \n sample_rate:{sample_rate}')

    # metadata_right = torchaudio.info(file_path_right)
    waveform_right, sample_rate = torchaudio.load(mono_right_path)
    # print(f'mono right \n waveform: {waveform_right} \n sample_rate:{sample_rate}')

    waveform_mean = (waveform_left + waveform_right) / 2

    sample_tuple = (waveform_mean, sample_rate, label, 'blah', 0)

    if i < split_percentage*num_files:
      train_set_list.append(sample_tuple)
    else:
      test_set_list.append(sample_tuple)

In [10]:
len(train_set_list), len(test_set_list)

(1298, 120)

In [11]:
labels

['baneshat',
 'ghamgin',
 ' pakar',
 ' khasmane',
 ' mashkuk',
 ' delnegaran',
 ' las zadan',
 'moshtagh',
 ' jeddi',
 ' alaghemand',
 ' motmaen',
 ' badgoman',
 'semej',
 'narahat',
 'delsard',
 ' shakkak',
 'delmashghul',
 ' pashiman',
 ' motevaghe',
 'negaran',
 ' mohtat',
 'khialardaz',
 ' bamolaheze',
 ' jasur',
 ' khungarm',
 ' dodel',
 ' mosammam',
 ' moraddad',
 ' motefakker',
 ' mottaham']

## Formatting the Data

This is a good place to apply transformations to the data. For the
waveform, we downsample the audio for faster processing without losing
too much of the classification power.

We don’t need to apply other transformations here. It is common for some
datasets though to have to reduce the number of channels (say from
stereo to mono) by either taking the mean along the channel dimension,
or simply keeping only one of the channels. Since SpeechCommands uses a
single channel for audio, this is not needed here.




In [12]:
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
transformed = transform(waveform_mean)

ipd.Audio(transformed.numpy(), rate=new_sample_rate)

We are encoding each word using its index in the list of labels.




In [13]:
######### Masoud cell #########
def label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(labels.index(word))


def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]


word_start = "baneshat"
index = label_to_index(word_start)
word_recovered = index_to_label(index)

print(word_start, "-->", index, "-->", word_recovered)

baneshat --> tensor(0) --> baneshat


To turn a list of data point made of audio recordings and utterances
into two batched tensors for the model, we implement a collate function
which is used by the PyTorch DataLoader that allows us to iterate over a
dataset by batches. Please see [the
documentation](https://pytorch.org/docs/stable/data.html#working-with-collate-fn)_
for more information about working with a collate function.

In the collate function, we also apply the resampling, and the text
encoding.




In [14]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets


batch_size = 256

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train_set_list,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
test_loader = torch.utils.data.DataLoader(
    test_set_list,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

## Define the Network

For this tutorial we will use a convolutional neural network to process
the raw audio data. Usually more advanced transforms are applied to the
audio data, however CNNs can be used to accurately process the raw data.
The specific architecture is modeled after the M5 network architecture
described in [this paper](https://arxiv.org/pdf/1610.00087.pdf)_. An
important aspect of models processing raw audio data is the receptive
field of their first layer’s filters. Our model’s first filter is length
80 so when processing audio sampled at 8kHz the receptive field is
around 10ms (and at 4kHz, around 20 ms). This size is similar to speech
processing applications that often use receptive fields ranging from
20ms to 40ms.




In [15]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=30, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)


model = M5(n_input=transformed.shape[0], n_output=len(labels))
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=30, bias=True)
)
Numbe

We will use the same optimization technique used in the paper, an Adam
optimizer with weight decay set to 0.0001. At first, we will train with
a learning rate of 0.01, but we will use a ``scheduler`` to decrease it
to 0.001 during training after 20 epochs.




In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10

## Training and Testing the Network

Now let’s define a training function that will feed our training data
into the model and perform the backward pass and optimization steps. For
training, the loss we will use is the negative log-likelihood. The
network will then be tested after each epoch to see how the accuracy
varies during the training.




In [17]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

Now that we have a training function, we need to make one for testing
the networks accuracy. We will set the model to ``eval()`` mode and then
run inference on the test dataset. Calling ``eval()`` sets the training
variable in all modules in the network to false. Certain layers like
batch normalization and dropout layers behave differently during
training so this step is crucial for getting correct results.




In [18]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")

Finally, we can train and test the network. We will train the network
for ten epochs then reduce the learn rate and train for ten more epochs.
The network will be tested after each epoch to see how the accuracy
varies during the training.




In [30]:
log_interval = 20
n_epoch = 400

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        test(model, epoch)
        scheduler.step()

# Let's plot the training loss versus the number of iteration.
# plt.plot(losses);
# plt.title("training loss");

  0%|          | 0.2857142857142857/400 [00:00<07:08,  1.07s/it] 

Train Epoch: 1 [0/1298 (0%)]	Loss: 2.753407


  0%|          | 1.1428571428571426/400 [00:01<05:41,  1.17it/s]


Test Epoch: 1	Accuracy: 10/120 (8%)

Train Epoch: 2 [0/1298 (0%)]	Loss: 2.866411


  1%|          | 2.1428571428571423/400 [00:01<05:06,  1.30it/s]

...


Test Epoch: 398	Accuracy: 10/120 (8%)

Train Epoch: 399 [0/1298 (0%)]	Loss: 2.814079


100%|█████████▉| 399.14285714287666/400 [06:48<00:00,  1.16it/s]


Test Epoch: 399	Accuracy: 11/120 (9%)

Train Epoch: 400 [0/1298 (0%)]	Loss: 2.861721


100%|█████████▉| 399.71428571430533/400 [06:48<00:00,  1.08s/it]/usr/local/lib/python3.10/dist-packages/tqdm/std.py:639: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 400.00000000001967/400 [06:48<00:00,  1.02s/it]


Test Epoch: 400	Accuracy: 10/120 (8%)



The network should be more than 65% accurate on the test set after 2
epochs, and 85% after 21 epochs. Let’s look at the last words in the
train set, and see how the model did on it.




In [31]:
def accuracy(data_loader):

  sum_acc = 0

  for batch_idx, (data, target) in enumerate(data_loader):

          num_correct_preds = 0
          num_wrong_preds = 0


          data = data.to(device)
          targets = target.to(device)

          data = data.to(device)
          # apply transform and model on whole batch directly on device
          data = transform(data)
          output = model(data)

          # print(f'model output: {output}')
          preds = get_likely_index(output)
          # pred_labels = index_to_label(tensor.squeeze())
          # print(f'model prediction: {preds}')
          # print(f'class label: {targets}')

          for (pred, target) in zip(preds, targets):
            if pred == target:
              num_correct_preds += 1
            else:
              num_wrong_preds += 1


          num_samples = len(preds)

          acc = num_correct_preds / num_samples
          sum_acc += acc

          # print(f'accuracy: {acc}')

  num_batches = batch_idx + 1
  acc_avg = sum_acc / num_batches

  return acc_avg

In [34]:
train_acc = accuracy(train_loader)
test_acc = accuracy(test_loader)

print(f'train acc: {train_acc} \n test acc: {test_acc}')

train acc: 0.16695601851851852 
 test acc: 0.08333333333333333
